# Initial Setup

## Libraries

In [38]:
import json
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import agent_tools

## Setups

In [33]:
# OpenAI Setup
load_dotenv(override=True)
openai = OpenAI()

# Pushover Setup
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [34]:
# The function will push message in the form of a payload to the phone based on predetermined parameters. 
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

# Development

In [35]:
# function if whether the user would like to be in touch with us
def record_user_details(email, name="name not provided.", notes='notes not provided.'):
    push(f"Recording interest from name: {name}, email: {email}, notes: {notes}")
    return {"recorded:": "ok"}

# function if the user doesn't know a question i doesn't know to answer. 
def record_unknown_question(question):
    push(f"Recording question: {question}")
    return {"recorded": "ok"}

In [ ]:
def handle_tool_calls(tool_calls):
    '''
    A function which takes in a list of tool calls, and executes them (sends a notification to phone) based on their prompt.
    Furthermore, each tool is already pre-mapped to a function, all this function does it search and execute like a dumb agent. 
    '''
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        # finds function in the program based on the tool name. 
        tool = globals()[tool_name]
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results